In [2]:
import pandas as pd
import numpy as np
import skrub

In [3]:
df1 = pd.DataFrame({"Country": ["USA", "Italy", "Georgia"],
                    "Person": ["P1", "P2", "P3"]})
df2 = pd.DataFrame({"Country": ["Italy", "Belgium", "Italy"],
                    "City": ["Palermo", "Brussel", "Rome"],
                    'population':[1000,2000,1000],
                    })

main_table = skrub.var("main_table", df1)
aux_table = skrub.var("aux_table", df2)


joined_right = (
    main_table
    .merge(
        aux_table,
        on="Country",
        how="outer",
    )
)
print(joined_right)
print(joined_right.groupby("Country").agg({"population": 'sum'}))

<CallMethod 'merge'>
Result:
―――――――
   Country Person     City  population
0  Belgium    NaN  Brussel      2000.0
1  Georgia     P3      NaN         NaN
2    Italy     P2  Palermo      1000.0
3    Italy     P2     Rome      1000.0
4      USA     P1      NaN         NaN
<CallMethod 'agg'>
Result:
―――――――
         population
Country            
Belgium      2000.0
Georgia         0.0
Italy        2000.0
USA             0.0


In [4]:
main_table

Column,Column name,dtype,Is sorted,Null values,Unique values,Mean,Std,Min,Median,Max
0,Country,ObjectDType,True,0 (0.0%),3 (100.0%),,,,,
1,Person,ObjectDType,True,0 (0.0%),3 (100.0%),,,,,


In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parents[1]
sys.path.insert(0, str(project_root))

print("Added to sys.path:", project_root)
import pandas as pd
import skrub
from monkey_patching_v02.data_provenance.monkey_patching_v02_data_provenance import enable_why_data_provenance, evaluate_provenance_fast, decode_prov_column
from monkey_patching_v02.data_provenance.provenance_utils_jeanne_performant import decode_prov
enable_why_data_provenance()

Added to sys.path: c:\Users\eduar\Documents\RDEPro_testing\rdepro_skrub_provenance


In [2]:
df1 = pd.DataFrame({"Country": ["USA", "Italy", "Georgia"],
                    "Person": ["P1", "P2", "P3"]})
df2 = pd.DataFrame({"Country": ["Italy", "Belgium", "Italy"],
                    "City": ["Palermo", "Brussel", "Rome"],
                    'population':[1000,2000,1000],
                    })

main_table = skrub.var("main_table", df1)
aux_table = skrub.var("aux_table", df2)


joined_right = (
    main_table
    .merge(
        aux_table,
        on="Country",
        how="outer",
    )
)
print(joined_right)
print(joined_right.groupby("Country").agg({"population": 'sum'}))

<CallMethod 'merge'>
Result:
―――――――
   Country Person  _prov0     City  population        _prov1
0  Belgium    NaN     NaN  Brussel      2000.0  2.814750e+14
1  Georgia     P3     2.0      NaN         NaN           NaN
2    Italy     P2     1.0  Palermo      1000.0  2.814750e+14
3    Italy     P2     1.0     Rome      1000.0  2.814750e+14
4      USA     P1     0.0      NaN         NaN           NaN
<CallMethod 'agg'>
Result:
―――――――
         population      _prov0                                  _prov1
Country                                                                
Belgium      2000.0       [nan]                     [281474976710657.0]
Georgia         0.0       [2.0]                                   [nan]
Italy        2000.0  [1.0, 1.0]  [281474976710656.0, 281474976710658.0]
USA             0.0       [0.0]                                   [nan]


In [3]:
decode_prov_column(joined_right)

KeyError: 1

In [4]:
final_prov_joined_right = evaluate_provenance_fast(joined_right)
final_prov_joined_right

,Country,Person,City,population,_prov
0,Belgium,NaN,Brussel,2000.0,{281474976710657}
1,Georgia,P3,NaN,NaN,{2}
2,Italy,P2,Palermo,1000.0,"{281474976710656, 1}"
3,Italy,P2,Rome,1000.0,"{1, 281474976710658}"
4,USA,P1,NaN,NaN,{0}


In [5]:
final_prov_joined_right["_prov"]

0       {281474976710657}
1                     {2}
2    {281474976710656, 1}
3    {1, 281474976710658}
4                     {0}
Name: _prov, dtype: object

In [7]:
final_prov_joined_right["_prov"].map(lambda set_x: [decode_prov(x) for x in set_x])

0                  [aux_table:1]
1                 [main_table:2]
2    [aux_table:0, main_table:1]
3    [main_table:1, aux_table:2]
4                 [main_table:0]
Name: _prov, dtype: object

In [ ]:
final_prov_joined_right["_prov"]

In [23]:
display(joined_right.assign(_prov0 = joined_right["_prov0"].fillna(-1).apply(int)).assign(_prov1=(joined_right["_prov1"].fillna(-1).apply(int)))[["Country", "City", "Person", "population", "_prov0", "_prov1"]])

Column,Column name,dtype,Is sorted,Null values,Unique values,Mean,Std,Min,Median,Max
0,Country,ObjectDType,True,0 (0.0%),4 (80.0%),,,,,
1,City,ObjectDType,False,2 (40.0%),3 (60.0%),,,,,
2,Person,ObjectDType,False,1 (20.0%),3 (60.0%),,,,,
3,population,Float64DType,False,2 (40.0%),2 (40.0%),1.33e+03,577.,1.00e+03,1.00e+03,2.00e+03
4,_prov0,Int64DType,False,0 (0.0%),4 (80.0%),0.600,1.14,-1,1,2
5,_prov1,Int64DType,False,0 (0.0%),4 (80.0%),1.69e+14,1.54e+14,-1,"281,474,976,710,656","281,474,976,710,658"


In [ ]:
display(joined_right[["Country", "City", "Person", "population", "_prov0", "_prov1"]])

Column,Column name,dtype,Is sorted,Null values,Unique values,Mean,Std,Min,Median,Max
0,Country,ObjectDType,True,0 (0.0%),4 (80.0%),,,,,
1,City,ObjectDType,False,2 (40.0%),3 (60.0%),,,,,
2,Person,ObjectDType,False,1 (20.0%),3 (60.0%),,,,,
3,population,Float64DType,False,2 (40.0%),2 (40.0%),1.33e+03,577.,1.00e+03,1.00e+03,2.00e+03
4,_prov0,Float64DType,False,1 (20.0%),3 (60.0%),1.00,0.816,0.00,1.00,2.00
5,_prov1,Float64DType,False,2 (40.0%),3 (60.0%),2.81e+14,1.00,2.81e+14,2.81e+14,2.81e+14


In [10]:
main_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  3 non-null      object
 1   Person   3 non-null      object
 2   _prov0   3 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 204.0+ bytes


c:\Users\eduar\AppData\Local\Programs\Python\Python311\Lib\site-packages\skrub\_data_ops\_data_ops.py:394: UserWarning: Calling '.info()' returned None. To enable chaining steps in a DataOps plan, do not use functions that modify objects in-place but rather functions that leave their argument unchanged and return a new object.
  warnings.warn(msg)


<CallMethod 'info'>
Result:
―――――――
None

In [14]:
main_table["_prov0"][0]

<GetItem 0>
Result:
―――――――
np.int64(0)

In [15]:
joined_right["_prov0"][0]

<GetItem 0>
Result:
―――――――
np.float64(nan)

In [12]:
main_table

Column,Column name,dtype,Is sorted,Null values,Unique values,Mean,Std,Min,Median,Max
0,Country,ObjectDType,True,0 (0.0%),3 (100.0%),,,,,
1,Person,ObjectDType,True,0 (0.0%),3 (100.0%),,,,,
2,_prov0,Int64DType,True,0 (0.0%),3 (100.0%),1.00,1.00,0,1,2


In [17]:
main_table.merge(
        aux_table,
        on="Country",
        how="inner",
    ).groupby("Country").agg({"population":"sum"})

Column,Column name,dtype,Is sorted,Null values,Unique values,Mean,Std,Min,Median,Max
0,population,Int64DType,True,0 (0.0%),1 (100.0%),2.00e+03,nan,,,
1,_prov0,ObjectDType,True,0 (0.0%),1 (100.0%),,,,,
2,_prov1,ObjectDType,True,0 (0.0%),1 (100.0%),,,,,


In [9]:
joined_right.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Country     5 non-null      object 
 1   Person      4 non-null      object 
 2   _prov0      4 non-null      float64
 3   City        3 non-null      object 
 4   population  3 non-null      float64
 5   _prov1      3 non-null      float64
dtypes: float64(3), object(3)
memory usage: 372.0+ bytes


c:\Users\eduar\AppData\Local\Programs\Python\Python311\Lib\site-packages\skrub\_data_ops\_data_ops.py:394: UserWarning: Calling '.info()' returned None. To enable chaining steps in a DataOps plan, do not use functions that modify objects in-place but rather functions that leave their argument unchanged and return a new object.
  warnings.warn(msg)


<CallMethod 'info'>
Result:
―――――――
None

In [ ]:
main_table.skb.select(["Country"])

Column,Column name,dtype,Is sorted,Null values,Unique values,Mean,Std,Min,Median,Max
0,Country,ObjectDType,True,0 (0.0%),3 (100.0%),,,,,
1,_prov0,Int64DType,True,0 (0.0%),3 (100.0%),1.00,1.00,0,1,2


In [11]:
joined_right.skb.select(["Country"])

Column,Column name,dtype,Is sorted,Null values,Unique values,Mean,Std,Min,Median,Max
0,Country,ObjectDType,True,0 (0.0%),4 (80.0%),,,,,
1,_prov0,Float64DType,False,1 (20.0%),3 (60.0%),1.00,0.816,0.00,1.00,2.00
2,_prov1,Float64DType,False,2 (40.0%),3 (60.0%),2.81e+14,1.00,2.81e+14,2.81e+14,2.81e+14


In [6]:
main_table

Column,Column name,dtype,Is sorted,Null values,Unique values,Mean,Std,Min,Median,Max
0,Country,ObjectDType,True,0 (0.0%),3 (100.0%),,,,,
1,Person,ObjectDType,True,0 (0.0%),3 (100.0%),,,,,
2,_prov0,Int64DType,True,0 (0.0%),3 (100.0%),1.00,1.00,0,1,2


In [8]:
display(joined_right.groupby("Country").agg({"population": 'sum'}))

Column,Column name,dtype,Is sorted,Null values,Unique values,Mean,Std,Min,Median,Max
0,population,Float64DType,False,0 (0.0%),2 (50.0%),1.00e+03,1.15e+03,0.00,2.00e+03,2.00e+03
1,_prov0,ObjectDType,False,0 (0.0%),4 (100.0%),,,,,
2,_prov1,ObjectDType,False,0 (0.0%),4 (100.0%),,,,,


In [24]:
df1 = pd.DataFrame({"Country": ["USA", "Italy", "Georgia"],
                    "Person": ["P1", "P2", "P3"]})
df2 = pd.DataFrame({"Country": ["Italy", "Belgium", "Italy"],
                    "City": ["Palermo", "Brussel", "Rome"],
                    'population':[1000,2000,1000],
                    })

main_table = skrub.var("main_table", df1)
aux_table = skrub.var("aux_table", df2)


joined_right = (
    main_table
    .merge(
        aux_table,
        on="Country",
        how="outer",
    )
)
print(joined_right)
ultimate_check = skrub.var("agg_df", joined_right.groupby("Country").skb.eval())
#print(joined_right.groupby("Country").agg({"population": 'sum'}))
ultimate_check

<CallMethod 'merge'>
Result:
―――――――
   Country Person  _prov0     City  population        _prov1
0  Belgium    NaN     NaN  Brussel      2000.0  2.814750e+14
1  Georgia     P3     2.0      NaN         NaN           NaN
2    Italy     P2     1.0  Palermo      1000.0  2.814750e+14
3    Italy     P2     1.0     Rome      1000.0  2.814750e+14
4      USA     P1     0.0      NaN         NaN           NaN


<Var 'agg_df'>
Result:
―――――――
<pandas.core.groupby.generic.DataFrameGroupBy object at 0x0000027A6A9AB690>

In [25]:
print(ultimate_check.agg({"population": 'sum'}))

<CallMethod 'agg'>
Result:
―――――――
         population      _prov0                                  _prov1
Country                                                                
Belgium      2000.0       [nan]                     [281474976710657.0]
Georgia         0.0       [2.0]                                   [nan]
Italy        2000.0  [1.0, 1.0]  [281474976710656.0, 281474976710658.0]
USA             0.0       [0.0]                                   [nan]


In [28]:
ultimate_check.assign(_prov33 = np.arange(len(ultimate_check.skb.eval())))
print(ultimate_check)

RuntimeError: Evaluation of '.assign' failed.
You can see the full traceback above. The error message was:
AttributeError: 'DataFrameGroupBy' object has no attribute 'assign'